In [1]:
! pip install gensim

In [2]:
! pip install pyLDAvis

In [3]:
! pip install spacy

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import scipy
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.multivariate.pca import PCA
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [5]:
train_df = pd.read_csv('WikiLarge_Train.csv')
test_df = pd.read_csv('WikiLarge_Test.csv')
train_df#['original_text']

,original_text,label
0,There is manuscript evidence that Austen conti...,1
1,"In a remarkable comparative analysis , Mandaea...",1
2,"Before Persephone was released to Hermes , who...",1
3,Cogeneration plants are commonly found in dist...,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1
...,...,...
416763,A Duke Nukem 3D version has been sold for Xbox...,0
416764,"However , it is becoming replaced as a method ...",0
416765,There are hand gestures in both Hindu and Budd...,0
416766,"If it is necessary to use colors , try to choo...",0


In [6]:
# much of the following work will be heavily influenced by the following instructional video: https://youtu.be/UEn3xHNBXJU?list=LL

# Using LDA to train supervised learning model

In [7]:
# saved to csv
def lemmatize_sentence(sentences, p_o_s=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    sentences_out = []
    for sentence in tqdm(sentences):
        doc = nlp(sentence)
        new_sent = []
        for token in doc:
            if token.pos_ in p_o_s:
                new_sent.append(token.lemma_)
        final = " ".join(new_sent)
        sentences_out.append(final)
    return(sentences_out)


lemmatized_sentences = lemmatize_sentence(train_df['original_text'])
print (lemmatized_sentences[0][0:50])

pd.DataFrame(lemmatized_sentences).to_csv('lemmatized_sentences.csv', index = False)

'def lemmatize_sentence(sentences, p_o_s=["NOUN", "ADJ", "VERB", "ADV"]):\n    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])\n    sentences_out = []\n    for sentence in tqdm(sentences):\n        doc = nlp(sentence)\n        new_sent = []\n        for token in doc:\n            if token.pos_ in p_o_s:\n                new_sent.append(token.lemma_)\n        final = " ".join(new_sent)\n        sentences_out.append(final)\n    return(sentences_out)\n\n\nlemmatized_sentences = lemmatize_sentence(train_df[\'original_text\'])\nprint (lemmatized_sentences[0][0:50])\n\npd.DataFrame(lemmatized_sentences).to_csv(\'lemmatized_sentences.csv\', index = False)'

In [8]:
lemmatized_sentences = pd.read_csv('lemmatized_sentences.csv')
lemmatized_sentences

,0
0,manuscript evidence continue work piece as lat...
1,remarkable comparative analysis mandaean schol...
2,release send retrieve trick eat pomegranate se...
3,cogeneration plant commonly find district heat...
4,second most populous city zürich most populous...
...,...
416763,3d version sell live
416764,however become replace method execute criminal...
416765,hand gesture iconography
416766,necessary use color try choose color will conf...


In [9]:
lemmatized_sentences['0']

0         manuscript evidence continue work piece as lat...
1         remarkable comparative analysis mandaean schol...
2         release send retrieve trick eat pomegranate se...
3         cogeneration plant commonly find district heat...
4         second most populous city zürich most populous...
                                ...                        
416763                                 3d version sell live
416764    however become replace method execute criminal...
416765                             hand gesture iconography
416766    necessary use color try choose color will conf...
416767                                                  NaN
Name: 0, Length: 416768, dtype: object

In [10]:
nan_df = lemmatized_sentences[lemmatized_sentences.isna().any(axis=1)]
nan_df

,0
31,NaN
90,NaN
104,NaN
140,NaN
145,NaN
...,...
416690,NaN
416694,NaN
416730,NaN
416745,NaN


In [11]:
nan_df2 = train_df[train_df.isna().any(axis=1)]
nan_df2

,original_text,label


In [12]:
print(lemmatized_sentences['0'][0][0:90])

manuscript evidence continue work piece as late period niece nephew make further addition 


In [13]:
type(lemmatized_sentences)

pandas.core.frame.DataFrame

In [14]:
lemmatized_sentences['0'][40:50]

40    bear american professional wrestler actor well...
41                 situate northern border coastline km
42    shade red orange may sometimes appear replace ...
43    also know name take newspaper mean clarion tak...
44    guitar traditionally construct various wood st...
45    player take role engineer name battle polymorp...
46             rez bear also know mexican racing driver
47         order book bind book click order book button
48    conflict parent religion mean most baptise bir...
49                           ndash german mathematician
Name: 0, dtype: object

In [15]:
train_df['original_text'][30:35]

30    On July 29 , 2008 the Palace announced that Ki...
31                               F. W. D. Brie , 1906 .
32    Simon Boccanegra is an opera with a prologue a...
33    On June 6 , 2000 , ten days before the movie w...
34    The Bushehr Nuclear Power Plant -LRB- -RRB- is...
Name: original_text, dtype: object

In [16]:
lemmatized_sentences = lemmatized_sentences.fillna('nan')

In [17]:
def gen_words(sentences):
    final = []
    for sentence in tqdm(sentences['0']):
        new = gensim.utils.simple_preprocess(sentence, deacc=True)
        final.append(new)
    return (final)

clean_words = gen_words(lemmatized_sentences)

print (clean_words[0][0:20])

100%|██████████| 416768/416768 [00:08<00:00, 47273.68it/s]

['manuscript', 'evidence', 'continue', 'work', 'piece', 'as', 'late', 'period', 'niece', 'nephew', 'make', 'further', 'addition', 'as', 'late']


# Bigrams and trigrams train set

In [18]:
bi_phrases = gensim.models.Phrases(clean_words, min_count=5, threshold=100)
tri_phrases = gensim.models.Phrases(bi_phrases[clean_words], threshold=100)
bigram = gensim.models.phrases.Phraser(bi_phrases)
trigram = gensim.models.phrases.Phraser(tri_phrases)

def make_bigrams(sentences):
    return([bigram[doc] for doc in sentences])
def make_trigrams(sentences):
    return ([trigram[bigram[doc]] for doc in sentences])

data_bigrams = make_bigrams(clean_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)
print (data_bigrams_trigrams[0][0:20])

['manuscript', 'evidence', 'continue', 'work', 'piece', 'as', 'late', 'period', 'niece', 'nephew', 'make', 'further', 'addition', 'as', 'late']


In [19]:
id2word = corpora.Dictionary(data_bigrams_trigrams)
sentences = data_bigrams_trigrams
corpus = [id2word.doc2bow(sentence) for sentence in sentences]
# print (corpus[0][0:20])
tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in tqdm(range(0, len(corpus))):
    bow = corpus[i]
    low_val_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_val_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_val_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_val_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

100%|██████████| 416768/416768 [00:09<00:00, 45426.28it/s]


In [20]:
# bi tri model saved
Bi_tri_lda_model = gensim.models.ldamodel.LdaModel(corpus=tqdm(corpus),
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=40,
                                           update_every=1,
                                           chunksize=80,
                                           passes=8,
                                           alpha="auto")
                                           
Bi_tri_lda_model.save("Bi_tri_lda_model.model")

'# bi tri model saved\nBi_tri_lda_model = gensim.models.ldamodel.LdaModel(corpus=tqdm(corpus),\n                                           id2word=id2word,\n                                           num_topics=20,\n                                           random_state=40,\n                                           update_every=1,\n                                           chunksize=80,\n                                           passes=8,\n                                           alpha="auto")\n                                           \nBi_tri_lda_model.save("Bi_tri_lda_model.model")'

In [21]:
Bi_tri_lda_model_saved_model = gensim.models.ldamodel.LdaModel.load("Bi_tri_lda_model.model")

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(Bi_tri_lda_model_saved_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.403108 -0.198353       1        1  9.628153
19    -0.488739  0.055339       2        1  6.523408
4     -0.058945 -0.192844       3        1  5.773782
16    -0.142226 -0.433951       4        1  5.552992
10     0.093899  0.463928       5        1  5.494105
12    -0.147129  0.158742       6        1  5.164802
11    -0.381132  0.248841       7        1  5.106026
9     -0.319255 -0.343935       8        1  5.095499
8      0.056272 -0.471644       9        1  5.018720
3      0.327812 -0.366991      10        1  4.939608
15     0.167861 -0.294761      11        1  4.899520
1      0.257014  0.373236      12        1  4.420342
6     -0.309258 -0.019618      13        1  4.345974
18     0.107631  0.108894      14        1  4.308096
17    -0.239889  0.405081      15        1  4.187696
0     -0.059381  0.397232      16        1  4.153211
13     0.277642 -0.113010      17        1  3.899681
7      0.458245 -0.104093      18        1  3.882072
5      0.439154  0.089204      19        1  3.842698
14     0.363530  0.238702      20        1  3.763614, topic_info=         Term          Freq         Total Category  logprob  loglift
271       nan  24691.000000  24691.000000  Default  30.0000  30.0000
343      bear  21237.000000  21237.000000  Default  29.0000  29.0000
363      also  21725.000000  21725.000000  Default  28.0000  28.0000
224       use  21359.000000  21359.000000  Default  27.0000  27.0000
6        make  19497.000000  19497.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
560   measure    776.042233    777.144881  Topic20  -4.9345   3.2784
3783    merge    736.698647    737.801296  Topic20  -4.9865   3.2783
3990     wave    705.518547    706.621195  Topic20  -5.0297   3.2782
2795      dry    599.971082    601.073730  Topic20  -5.1918   3.2780
5261    cloud    561.493859    562.596507  Topic20  -5.2581   3.2778

[630 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
812      18  0.999672     about
21       11  0.999513    accord
272      13  0.998935       act
1280      9  0.999056  activity
342      11  0.999679     actor
...     ...       ...       ...
270       3  0.999804     would
121      14  0.999856     write
315      15  0.999301    writer
33        8  0.999922      year
1835      4  0.998988     young

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 20, 5, 17, 11, 13, 12, 10, 9, 4, 16, 2, 7, 19, 18, 1, 14, 8, 6, 15])

In [23]:
corpus[400001]

[(39, 1), (644, 1), (1634, 1), (2523, 1), (4551, 1)]

In [24]:
train_df['original_text'][400001]

"They decide to find Shade 's father , Cassiel , and Chinook volunteers to help ."

# Get bag of words for test df

In [25]:
# get back to this if test split returns high accuracy

test_sentence = ['This', 'is', 'a', 'test']
id2word.doc2bow(test_sentence)

[(2386, 1), (21950, 1)]

In [26]:
# saved to csv
def lemmatize_sentence(sentences, p_o_s=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    sentences_out = []
    for sentence in tqdm(sentences):
        doc = nlp(sentence)
        new_sent = []
        for token in doc:
            if token.pos_ in p_o_s:
                new_sent.append(token.lemma_)
        final = " ".join(new_sent)
        sentences_out.append(final)
    return(sentences_out)


lemmatized_sentences = lemmatize_sentence(test_df['original_text'])
print (lemmatized_sentences[0][0:50])

pd.DataFrame(lemmatized_sentences).to_csv('lemmatized_test_sentences.csv', index = False)

'def lemmatize_sentence(sentences, p_o_s=["NOUN", "ADJ", "VERB", "ADV"]):\n    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])\n    sentences_out = []\n    for sentence in tqdm(sentences):\n        doc = nlp(sentence)\n        new_sent = []\n        for token in doc:\n            if token.pos_ in p_o_s:\n                new_sent.append(token.lemma_)\n        final = " ".join(new_sent)\n        sentences_out.append(final)\n    return(sentences_out)\n\n\nlemmatized_sentences = lemmatize_sentence(test_df[\'original_text\'])\nprint (lemmatized_sentences[0][0:50])\n\npd.DataFrame(lemmatized_sentences).to_csv(\'lemmatized_test_sentences.csv\', index = False)'

In [27]:
lemmatized_test_sentences = pd.read_csv('lemmatized_test_sentences.csv')
lemmatized_test_sentences = lemmatized_test_sentences.fillna('nan')

In [28]:
def gen_words(sentences):
    final = []
    for sentence in tqdm(sentences['0']):
        new = gensim.utils.simple_preprocess(sentence, deacc=True)
        final.append(new)
    return (final)

clean_words = gen_words(lemmatized_test_sentences)

print(clean_words[1][0:20])

100%|██████████| 119092/119092 [00:02<00:00, 50445.08it/s]

['nan']


# Bigrams and trigrams test df
Test DF code not used because the accuracy was too low with the train df train test split

In [29]:
# get back to this if test split returns high accuracy
'''#BIGRAMS AND TRIGRAMS
bi_phrases = gensim.models.Phrases(clean_words, min_count=5, threshold=100)
tri_phrases = gensim.models.Phrases(bi_phrases[clean_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bi_phrases)
trigram = gensim.models.phrases.Phraser(tri_phrases)

def make_bigrams(sentences):
    return([bigram[doc] for doc in sentences])

def make_trigrams(sentences):
    return ([trigram[bigram[doc]] for doc in sentences])

data_bigrams = make_bigrams(clean_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])'''

'#BIGRAMS AND TRIGRAMS\nbi_phrases = gensim.models.Phrases(clean_words, min_count=5, threshold=100)\ntri_phrases = gensim.models.Phrases(bi_phrases[clean_words], threshold=100)\n\nbigram = gensim.models.phrases.Phraser(bi_phrases)\ntrigram = gensim.models.phrases.Phraser(tri_phrases)\n\ndef make_bigrams(sentences):\n    return([bigram[doc] for doc in sentences])\n\ndef make_trigrams(sentences):\n    return ([trigram[bigram[doc]] for doc in sentences])\n\ndata_bigrams = make_bigrams(clean_words)\ndata_bigrams_trigrams = make_trigrams(data_bigrams)\n\nprint (data_bigrams_trigrams[0][0:20])'

In [54]:
# get back to this if test split returns high accuracy
'''#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

sentences = data_bigrams_trigrams

corpus = [id2word.doc2bow(sentence) for sentence in sentences]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_val_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_val_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_val_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_val_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow'''

'#TF-IDF REMOVAL\nfrom gensim.models import TfidfModel\n\nid2word = corpora.Dictionary(data_bigrams_trigrams)\n\nsentences = data_bigrams_trigrams\n\ncorpus = [id2word.doc2bow(sentence) for sentence in sentences]\n# print (corpus[0][0:20])\n\ntfidf = TfidfModel(corpus, id2word=id2word)\n\nlow_value = 0.03\nwords  = []\nwords_missing_in_tfidf = []\nfor i in range(0, len(corpus)):\n    bow = corpus[i]\n    low_val_words = []\n    tfidf_ids = [id for id, value in tfidf[bow]]\n    bow_ids = [id for id, value in bow]\n    low_val_words = [id for id, value in tfidf[bow] if value < low_value]\n    drops = low_val_words+words_missing_in_tfidf\n    for item in drops:\n        words.append(id2word[item])\n    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]\n\n    new_bow = [b for b in bow if b[0] not in low_val_words and b[0] not in words_missing_in_tfidf]\n    corpus[i] = new_bow'

In [31]:
for x in range(300):
    print(clean_words[x][0:20])

['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['feb']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['direct', 'video', 'sequel', 'film']
['fact', 'excommunicate', 'leave', 'second', 'time', 'result', 'victory', 'mark', 'end']
['tv', 'series', 'base', 'famous', 'video', 'game', 'character']
['nan']
['shall', 'issue', 'probable', 'cause', 'support', 'affirmation', 'particularly', 'describe', 'place', 'search', 'person', 'thing', 'seize']
['represent', 'occasion', 'require', 'prompt', 'attendance', 'spot', 'keep', 'study', 'ignorance', 'part', 'concerted', 'national', 'rising', 'still', 'look', 'submissive', 'vassal']
['represent', 'occasion', 'require', 'prompt', 'attendance', 'spot', 'keep', 'study', 'ignorance', 'part', 'concerted', 'national', 'rising', 'still', 'look', 'submissive', 'vassal']
['home']
['life', 'alternativ

In [32]:
#don't use seperately for test set
'''id2word = corpora.Dictionary(clean_words)

corpus = []
for sentence in tqdm(clean_words):
    new = id2word.doc2bow(sentence)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)'''

'id2word = corpora.Dictionary(clean_words)\n\ncorpus = []\nfor sentence in tqdm(clean_words):\n    new = id2word.doc2bow(sentence)\n    corpus.append(new)\n\nprint (corpus[0][0:20])\n\nword = id2word[[0][:1][0]]\nprint(word)'

In [33]:
clean_words[206]

['song',
 'perform',
 'american',
 'rapper',
 'hip',
 'hop',
 'artist',
 'feature',
 'vocal',
 'soul',
 'singer']

In [34]:
doc_bow_test = id2word.doc2bow(clean_words[30])
doc_lda_test = Bi_tri_lda_model_saved_model[doc_bow_test]
doc_lda_test[0][1]

0.06958477

In [35]:
for x in range(10):
    print(Bi_tri_lda_model_saved_model.get_document_topics(corpus, per_word_topics=True)[x][0])

[(0, 0.032189965), (1, 0.03154046), (2, 0.068925306), (3, 0.053861927), (4, 0.05969587), (5, 0.027253583), (6, 0.03106805), (7, 0.027563699), (8, 0.09242711), (9, 0.036161833), (10, 0.04065343), (11, 0.036830444), (12, 0.056090288), (13, 0.027661758), (14, 0.026802726), (15, 0.053541884), (16, 0.05924226), (17, 0.16203508), (18, 0.030801607), (19, 0.04565275)]
[(0, 0.03711057), (1, 0.05819915), (2, 0.07946133), (3, 0.06209534), (4, 0.046983704), (5, 0.031419605), (6, 0.057654522), (7, 0.14096394), (8, 0.04104357), (9, 0.041689582), (10, 0.046867773), (11, 0.042460397), (12, 0.042826965), (13, 0.031890176), (14, 0.03089983), (15, 0.03988901), (16, 0.046460755), (17, 0.033942446), (18, 0.03550999), (19, 0.0526313)]
[(0, 0.033562277), (1, 0.07411271), (2, 0.07330947), (3, 0.056195088), (4, 0.0483704), (5, 0.028458066), (6, 0.03243645), (7, 0.02879704), (8, 0.037039686), (9, 0.058377292), (10, 0.04281009), (11, 0.038880367), (12, 0.039152898), (13, 0.028890517), (14, 0.02803424), (15, 0.17

# Text vector for train data

In [36]:
len(train_df)

416768

In [37]:
topic_count = 20

for y in range(topic_count):
    train_df['sentence_vectors_'+str(y)] = .00001

for x in tqdm(range(100000)):
    for y in range(topic_count):
        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]
        
train_df.to_csv('train_df_bi_tri_vectors_1.csv', index = False)

"topic_count = 20\n\nfor y in range(topic_count):\n    train_df['sentence_vectors_'+str(y)] = .00001\n\nfor x in tqdm(range(100000)):\n    for y in range(topic_count):\n        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]\n        \ntrain_df.to_csv('train_df_bi_tri_vectors_1.csv', index = False)"

In [38]:
topic_count = 20

for y in range(topic_count):
    train_df['sentence_vectors_'+str(y)] = .00001

for x in tqdm(range(100000,200000)):
    for y in range(topic_count):
        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]
        
train_df.to_csv('train_df_bi_tri_vectors_2.csv', index = False)

"topic_count = 20\n\nfor y in range(topic_count):\n    train_df['sentence_vectors_'+str(y)] = .00001\n\nfor x in tqdm(range(100000,200000)):\n    for y in range(topic_count):\n        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]\n        \ntrain_df.to_csv('train_df_bi_tri_vectors_2.csv', index = False)"

In [39]:
topic_count = 20

for y in range(topic_count):
    train_df['sentence_vectors_'+str(y)] = .00001

for x in tqdm(range(200000,300000)):
    for y in range(topic_count):
        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]
        
train_df.to_csv('train_df_bi_tri_vectors_3.csv', index = False)

"topic_count = 20\n\nfor y in range(topic_count):\n    train_df['sentence_vectors_'+str(y)] = .00001\n\nfor x in tqdm(range(200000,300000)):\n    for y in range(topic_count):\n        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]\n        \ntrain_df.to_csv('train_df_bi_tri_vectors_3.csv', index = False)"

In [40]:
topic_count = 20

for y in range(topic_count):
    train_df['sentence_vectors_'+str(y)] = .00001

for x in tqdm(range(300000,len(train_df))):
    for y in range(topic_count):
        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]
        
train_df.to_csv('train_df_bi_tri_vectors_4.csv', index = False)

"topic_count = 20\n\nfor y in range(topic_count):\n    train_df['sentence_vectors_'+str(y)] = .00001\n\nfor x in tqdm(range(300000,len(train_df))):\n    for y in range(topic_count):\n        train_df['sentence_vectors_'+str(y)][x] = Bi_tri_lda_model_saved_model.get_document_topics(corpus)[x][y][1]\n        \ntrain_df.to_csv('train_df_bi_tri_vectors_4.csv', index = False)"

In [41]:
#split up for loading issues. Utilizing great lakes did not speed up the processing power of gensim. Notebooks in
#Great Lakes and Coursera kept failing.

train_df_bi_tri_vectors_1 = pd.read_csv('train_df_bi_tri_vectors_1.csv')
train_df_bi_tri_vectors_2 = pd.read_csv('train_df_bi_tri_vectors_2.csv')
train_df_bi_tri_vectors_3 = pd.read_csv('train_df_bi_tri_vectors_3.csv')
train_df_bi_tri_vectors_4 = pd.read_csv('train_df_bi_tri_vectors_4.csv')

frames = [train_df_bi_tri_vectors_1[0:100000], train_df_bi_tri_vectors_2[100000:200000], train_df_bi_tri_vectors_3[200000:300000], train_df_bi_tri_vectors_4[300000:]]
train_df_bi_tri_vectors = pd.concat(frames)
train_df_bi_tri_vectors

,original_text,label,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,sentence_vectors_7,...,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
0,There is manuscript evidence that Austen conti...,1,0.032190,0.031540,0.068925,0.053862,0.059696,0.027254,0.031068,0.027564,...,0.040653,0.036830,0.056090,0.027662,0.026803,0.053542,0.059242,0.162035,0.030802,0.045653
1,"In a remarkable comparative analysis , Mandaea...",1,0.037111,0.058199,0.079461,0.062095,0.046984,0.031420,0.057655,0.140964,...,0.046868,0.042460,0.042827,0.031890,0.030900,0.039889,0.046461,0.033942,0.035510,0.052631
2,"Before Persephone was released to Hermes , who...",1,0.033562,0.074113,0.073309,0.056195,0.048367,0.028458,0.032436,0.028797,...,0.042810,0.038881,0.039153,0.028891,0.028034,0.175085,0.041891,0.050654,0.032125,0.051821
3,Cogeneration plants are commonly found in dist...,1,0.028594,0.045670,0.061359,0.041469,0.038020,0.058981,0.028009,0.024975,...,0.052748,0.066521,0.034926,0.232206,0.024089,0.048113,0.035455,0.027248,0.027884,0.040745
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1,0.036368,0.035617,0.077071,0.039370,0.045866,0.030845,0.035132,0.031208,...,0.088666,0.041521,0.041864,0.031300,0.030373,0.039017,0.152504,0.033276,0.034803,0.051296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416763,A Duke Nukem 3D version has been sold for Xbox...,0,0.041659,0.065333,0.089201,0.045192,0.052742,0.035271,0.040207,0.060186,...,0.052612,0.047665,0.048076,0.035799,0.034687,0.069292,0.052155,0.038103,0.039862,0.059082
416764,"However , it is becoming replaced as a method ...",0,0.037939,0.037286,0.092645,0.063482,0.048975,0.054446,0.059083,0.082415,...,0.047916,0.043409,0.043783,0.032602,0.031590,0.040780,0.073071,0.034700,0.036303,0.054106
416765,There are hand gestures in both Hindu and Budd...,0,0.042634,0.041708,0.090511,0.046131,0.053824,0.036085,0.041194,0.063919,...,0.053751,0.075271,0.049104,0.036609,0.035557,0.045719,0.053202,0.038946,0.040760,0.060210
416766,"If it is necessary to use colors , try to choo...",0,0.028905,0.028322,0.078900,0.031356,0.087621,0.058490,0.044907,0.024751,...,0.036505,0.050081,0.033357,0.024839,0.024067,0.048078,0.070205,0.077463,0.044667,0.040994


In [42]:
train_df_bi_tri_vectors = pd.read_csv('train_df_bi_tri_vectors.csv')

train_df_bi_tri_vectors

"train_df_bi_tri_vectors = pd.read_csv('train_df_bi_tri_vectors.csv')\n\ntrain_df_bi_tri_vectors"

In [43]:
#Bi_tri_lda_model_saved_model.get_document_topics(corpus, per_word_topics=False)[0]#[0][0][1]


In [44]:
#Bi_tri_lda_model_saved_model.get_document_topics(corpus, per_word_topics=True)[0]#[0][0][1]

In [45]:
#doc_bow_test = id2word.doc2bow(clean_words[50])
#doc_lda_test = Bi_tri_lda_model_saved_model[doc_bow_test]
#doc_lda_test[0][1][1]

# Text vector for test df
Test DF code not used because the accuracy was too low with the train df train test split

In [46]:
# saved to csv
'''topic_count = 20

for y in range(topic_count):
    test_df['sentence_vectors_'+str(y)] = .01

for x in tqdm(range(len(test_df))):
    doc_bow_test = id2word.doc2bow(clean_words[x])
    doc_lda_test = saved_model[doc_bow_test]        
    for y in range(topic_count):
        test_df['sentence_vectors_'+str(y)][x] = doc_lda_test[y][1]
        
test_df.to_csv('test_df_vectors.csv', index = False)'''

"topic_count = 20\n\nfor y in range(topic_count):\n    test_df['sentence_vectors_'+str(y)] = .01\n\nfor x in tqdm(range(len(test_df))):\n    doc_bow_test = id2word.doc2bow(clean_words[x])\n    doc_lda_test = saved_model[doc_bow_test]        \n    for y in range(topic_count):\n        test_df['sentence_vectors_'+str(y)][x] = doc_lda_test[y][1]\n        \ntest_df.to_csv('test_df_vectors.csv', index = False)"

In [47]:
'''test_df_vectors = pd.read_csv('test_df_bi_tri_vectors.csv')

test_df_vectors'''

"test_df_vectors = pd.read_csv('test_df_bi_tri_vectors.csv')\n\ntest_df_vectors"

# Text vector for test df bigrams trigrams
Test DF code not used because the accuracy was too low with the train df train test split

In [48]:
# saved to csv
'''topic_count = 20

for y in range(topic_count):
    test_df['sentence_vectors_'+str(y)] = .01

for x in tqdm(range(len(test_df))):
    doc_bow_test = id2word.doc2bow(data_bigrams_trigrams[x])
    doc_lda_test = Bi_tri_lda_model_saved_model[doc_bow_test]        
    for y in range(topic_count):
        test_df['sentence_vectors_'+str(y)][x] = doc_lda_test[0][y][1] #added the zero
        
test_df.to_csv('test_df_bigrams_trigrams_vectors.csv', index = False)'''

"topic_count = 20\n\nfor y in range(topic_count):\n    test_df['sentence_vectors_'+str(y)] = .01\n\nfor x in tqdm(range(len(test_df))):\n    doc_bow_test = id2word.doc2bow(data_bigrams_trigrams[x])\n    doc_lda_test = Bi_tri_lda_model_saved_model[doc_bow_test]        \n    for y in range(topic_count):\n        test_df['sentence_vectors_'+str(y)][x] = doc_lda_test[0][y][1] #added the zero\n        \ntest_df.to_csv('test_df_bigrams_trigrams_vectors.csv', index = False)"

In [49]:
test_df_bigrams_trigrams_vectors = pd.read_csv('test_df_bigrams_trigrams_vectors.csv')

test_df_bigrams_trigrams_vectors

,id,original_text,label,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,...,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
0,0,-2011,NaN,0.069585,0.042923,0.093800,0.047522,0.055462,0.037089,0.042280,...,0.055325,0.050122,0.050555,0.037645,0.036476,0.047087,0.054844,0.040067,0.041918,0.062128
1,1,-2011,NaN,0.069585,0.042923,0.093800,0.047522,0.055462,0.037089,0.042280,...,0.055325,0.050122,0.050555,0.037645,0.036476,0.047087,0.054844,0.040067,0.041918,0.062128
2,2,-2000,NaN,0.069585,0.042923,0.093800,0.047522,0.055462,0.037089,0.042280,...,0.055325,0.050122,0.050555,0.037645,0.036476,0.047087,0.054844,0.040067,0.041918,0.062128
3,3,-1997,NaN,0.069585,0.042923,0.093800,0.047522,0.055462,0.037089,0.042280,...,0.055325,0.050122,0.050555,0.037645,0.036476,0.047087,0.054844,0.040067,0.041918,0.062128
4,4,1.636,NaN,0.069585,0.042923,0.093800,0.047522,0.055462,0.037089,0.042280,...,0.055325,0.050122,0.050555,0.037645,0.036476,0.047087,0.054844,0.040067,0.041918,0.062128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119087,119087,#NAME?,NaN,0.044966,0.044059,0.096282,0.048780,0.056929,0.038070,0.043399,...,0.056789,0.051448,0.051893,0.038641,0.037441,0.048333,0.056296,0.041127,0.043027,0.063772
119088,119088,#NAME?,NaN,0.044966,0.044059,0.096282,0.048780,0.056929,0.038070,0.043399,...,0.056789,0.051448,0.051893,0.038641,0.037441,0.048333,0.056296,0.041127,0.043027,0.063772
119089,119089,#NAME?,NaN,0.044966,0.044059,0.096282,0.048780,0.056929,0.038070,0.043399,...,0.056789,0.051448,0.051893,0.038641,0.037441,0.048333,0.056296,0.041127,0.043027,0.063772
119090,119090,#NAME?,NaN,0.044966,0.044059,0.096282,0.048780,0.056929,0.038070,0.043399,...,0.056789,0.051448,0.051893,0.038641,0.037441,0.048333,0.056296,0.041127,0.043027,0.063772


# Logistic Regression

In [50]:
train_df_bi_tri_vectors.iloc[[180000]]

,original_text,label,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,sentence_vectors_7,...,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
180000,Berre-les-Alpes is a commune in the Alpes-Mari...,1,0.090687,0.040819,0.089201,0.045192,0.052742,0.035271,0.040207,0.035672,...,0.052612,0.047665,0.048076,0.035799,0.034687,0.044778,0.052155,0.062617,0.039862,0.059082


In [51]:

X = train_df_bi_tri_vectors[['sentence_vectors_0',
                    'sentence_vectors_1',
                    'sentence_vectors_2',
                    'sentence_vectors_3',
                    'sentence_vectors_4',
                    'sentence_vectors_5',
                    'sentence_vectors_6',
                    'sentence_vectors_7',
                    'sentence_vectors_8',
                    'sentence_vectors_9',
                    'sentence_vectors_10',
                    'sentence_vectors_11',
                    'sentence_vectors_12',
                    'sentence_vectors_13',
                    'sentence_vectors_14',
                    'sentence_vectors_15',
                    'sentence_vectors_16',
                    'sentence_vectors_17',
                    'sentence_vectors_18',
                    'sentence_vectors_19',]]
y = train_df_bi_tri_vectors['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train

,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,sentence_vectors_7,sentence_vectors_8,sentence_vectors_9,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
139689,0.039358,0.038447,0.084596,0.051665,0.073120,0.033227,0.037909,0.056721,0.069944,0.044219,0.049770,0.045073,0.045395,0.033777,0.052607,0.042254,0.049309,0.035827,0.060830,0.055918
365169,0.061640,0.038022,0.083090,0.042097,0.049129,0.055689,0.037453,0.033228,0.042918,0.043594,0.049008,0.067234,0.044783,0.033347,0.032311,0.087380,0.048583,0.035493,0.037132,0.077870
166281,0.042802,0.041914,0.090515,0.046315,0.053939,0.036316,0.041501,0.036747,0.047202,0.073297,0.053843,0.048846,0.049245,0.036851,0.035769,0.045903,0.078562,0.039167,0.040962,0.060303
38458,0.064590,0.039842,0.087067,0.044111,0.051480,0.034427,0.063173,0.034819,0.068899,0.069607,0.051353,0.046524,0.046926,0.034942,0.033857,0.043707,0.050908,0.037191,0.038909,0.057669
317952,0.041659,0.040819,0.089201,0.069706,0.077256,0.035271,0.040207,0.035672,0.046074,0.046799,0.077126,0.047665,0.048076,0.035799,0.034687,0.044778,0.052155,0.038103,0.039862,0.059082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93939,0.029405,0.028812,0.062962,0.031899,0.071834,0.094108,0.028380,0.025179,0.084431,0.033033,0.054439,0.050947,0.051238,0.059875,0.041787,0.031607,0.071420,0.044198,0.062743,0.041703
131621,0.039712,0.062279,0.085032,0.066449,0.050277,0.033622,0.038328,0.034005,0.043921,0.044612,0.050153,0.045437,0.045829,0.034126,0.033066,0.066054,0.073086,0.036322,0.037999,0.079689
276246,0.034129,0.053524,0.093161,0.057107,0.043209,0.028896,0.032940,0.029224,0.037746,0.058424,0.063186,0.039049,0.099636,0.029328,0.028418,0.056768,0.042728,0.031216,0.072824,0.068486
198410,0.040739,0.039895,0.086214,0.044089,0.051352,0.034561,0.039360,0.059293,0.044934,0.045668,0.099315,0.046499,0.046880,0.035070,0.034037,0.043696,0.050781,0.037278,0.038987,0.081351


In [52]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score


clf = LogisticRegression(solver='lbfgs', multi_class='auto')
#clf.fit(X_train[0:10000], y_train[0:10000])
clf.fit(X_train, y_train)

#X_dev = vectorizer.transform(dev_df['bio'])
#y_dev = list(dev_df.nationality)

# dummy classifier for testing purposes
dummy_clf_uniform = DummyClassifier(strategy="uniform")
dummy_clf_uniform.fit(X_train, y_train)

dummy_clf_frequent = DummyClassifier(strategy="most_frequent")
dummy_clf_frequent.fit(X_train, y_train)

lr_tiny_preds = clf.predict(X_test)
rand_preds = dummy_clf_uniform.predict(X_test)
mf_preds = dummy_clf_frequent.predict(X_test)

lr_f1 = f1_score(y_test, lr_tiny_preds, average='macro')
rand_f1 = f1_score(y_test, rand_preds, average='macro')
mf_f1 = f1_score(y_test, mf_preds, average='macro')

print(lr_f1)
print(rand_f1)
print(mf_f1)

0.5369434520728733
0.4995641475916742
0.33236362704327155


In [53]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping( monitor='val_loss', patience=30 )
lr_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=.5, patience=10, verbose=1 )

model.fit(X_train, y_train, epochs=10)

model.evaluate(X_test,  y_test, verbose=2)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

9768/9768 [==============================] - 7s 681us/step - loss: 0.7047 - accuracy: 0.5253
Epoch 2/10
9768/9768 [==============================] - 6s 662us/step - loss: 0.6786 - accuracy: 0.5743
Epoch 3/10
9768/9768 [==============================] - 6s 662us/step - loss: 0.6728 - accuracy: 0.5813
Epoch 4/10
9768/9768 [==============================] - 7s 698us/step - loss: 0.6693 - accuracy: 0.5857
Epoch 5/10
9768/9768 [==============================] - 7s 729us/step - loss: 0.6674 - accur

[0.6623764038085938, 0.6023495197296143]